## Authentication with OAuth2

Some of you may have already used APIs that require passing a specific key, to authorize the API calls. So the question naturally comes up: Why is that not sufficient for authentication?

Here are a few reasons:

* We often want our application to **act on behalf of a user** (e.g., retrieve the list of friends of a user on Facebook, and do some analysis on behalf of the user). OAuth allows for such delegation, without requiring the app to have access to the login credentials of the user.
* Acting on behalf of a user also allows the quota to be adjusted on a per-user basis, as opposed to a per-app basis. (So that the creators of very popular apps do not have to increase the quota for their own key)
* Users often want to give limited set of priviledges to the app (e.g., read only my profile, no posting).
* Users want to be able to selectively remove access for specific apps, without having to change the credentials for other apps.

So, how does OAuth achieves that?

## OAuth2 flows

Fundamentally, we have the following steps:

* The app sends the user to a login page. The login page asks the user whether the user really wants to grant these permissions to the app.
* The user logs in and grants the permissions. This generates an **authorization code** that the API sends back to the app (by **calling back a _redirect URL_**)
* The app uses the authorization code (which proves that the user has granted permissions), and calls the API, asking for an **access token**.
* The **access token** can then be used by the app to call the API on behalf of the user.

The picture below illustrates the OAuth2 flow:

<img src="https://assets.digitalocean.com/articles/oauth/abstract_flow.png">

Also these two tutorials explain in a simplified manner the details of the authentication process:
* https://www.digitalocean.com/community/tutorials/an-introduction-to-oauth-2
* http://aaronparecki.com/articles/2012/07/29/1/oauth2-simplified



### Running our own Python-based web server: Flask

As you can see from the above "flow", in OAuth2 authentication, we will need to have our own web server, to receive the answers that come back from the API. Python can do that using Flask, a very lightweight webserver. Below you can find a small example of a webserver that will run on your machine at port 5000. 

_Please note that we have configured this webserver to shutdown after a responding to a single call (we added the "stop_server()" function call in call responses)._

Once you run the call, visit http://yourIP:5000/ or http://yourIP:5000/testNYU to see the results. The server will stop immediately after, and the cell will stop being in "Busy" status.

In [ ]:
# In this cell, we configure our example web server. 
# We will show in the next cell how to configure this
# web server to handle the OAuth2 calls.

# Flask is a webserver
from flask import Flask, request

# Initialize the embedded web server
webserver = Flask("MyFirstWebServer")

# Go to http://<your IP>:5000/ to see the response
@webserver.route('/')
def hello_world():
    stop_server()
    return 'Hello World!'

# Go to http://<your IP>:5000/testNYU to see the different message
@webserver.route('/testNYU')
def hello_nyu():
    stop_server()
    return 'Hello NYU!'

def start_server():
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return

start_server()

### Accessing the Jawbone API using OAuth2

OK, enough with the example. 

Next, we will create the code that can interact with the Jawbone API. The Jawbone API documentation (https://jawbone.com/up/developer/) provides a [good explanation of the authentication process](https://jawbone.com/up/developer/authentication). 

Below, we implement the OAuth2 flow for the API. The code below will create the webserver that will handle the "callback" calls from the API. (Steps 3 and 4 from the figure above.) Then, we will show how to implement the rest of the flow.

In [1]:
# This part creates a temporary web server to "receive" the requests by Jawbone

# In this cell, we just configure the server. We will start it, 
# using the command webserver.run(host='0.0.0.0') in the next cell

# Flask is a webserver
from flask import Flask, request

# We will use that part to issue a request to Jawbone
import requests
import json



# Get these after registering an app with Jawbone
# These are to communicate to Jawbone that the requests come
# from a legitimate, registered app
CLIENT_ID = 'F3rPnz_fTUY'
CLIENT_SECRET = 'efaedc99a6bc5fb40f4be3b8fb97c6b94a7eec58'

# This is the location where we will store the authentication data from Jawbone
OAUTH_FILE = '/home/ubuntu/data/jawbone_secret.json'

# Initialize the embedded web server
webserver = Flask("JawboneOAuth")

# The function below will implement the sole functionality of our server.
# The Jawbone API will first authenticate the user and then direct the browser
# to http://<your ip>:5000/receiveCode in order to send the code to the app.
# 
# This is the place where the webserver will receive the call from Jawbone
# with the authentication code (Step 3 in https://jawbone.com/up/developer/authentication)
# Notice that we will use the http://<your ip>:5000/receiveCode address below as the redirect URI 
@webserver.route("/receiveCode")
def oauth_helper():
    code = request.args.get('code')
    
    # Now that we got the code, we move to Step 4 of the instructions
    # and request the access token from Jawbone. Notice that we 
    # use the client_secret to prove that the app is the real one
    # that was registered with the Jawbone API
    url = "https://jawbone.com/auth/oauth2/token"
    params = {"grant_type": "authorization_code", 
              "client_id": CLIENT_ID, 
              "client_secret": CLIENT_SECRET, 
              "code": code }
    resp = requests.get(url, params=params)
    data = json.loads(resp.text)
    
    # We store the code in a file as the webserver does not interact with the 
    # rest of the Python code, and we also want to reuse the code in the future
    # (Typically, we would store the access_token in a database.)
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(resp.text)
    f.close()
    
    # It is safe (and convenient) to shut down the web server after this request
    stop_server()
    
    # What we return here has no real impact on the functionality of the code
    return '<html><body>Code: <b>'+code+'</b><p>Response:<b>'+resp.text+'</b></body></html>'


def start_server():
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return
    


So our server is running now, and it's ready to accept the requests from Jawbone. 

Let's start the OAuth2 flow now, to first interact with the user and get the user to login and grant permissions. (Steps 1 and 2 of the figure above.)

In [2]:
# To get the server to run in the background
import threading

# This part is required to come up with the pop-up with which Jawbone will ask for permissions
from IPython.display import display
from IPython.display import Javascript as JS


# Send an OAuth request to Jawbone, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
REDIRECT_URI = 'http://ipython.ipeirotis.com:5000/receiveCode'
JAWBONE_URL = 'https://jawbone.com/auth/oauth2/auth'
PERMISSIONS = 'basic_read extended_read sleep_read move_read'

url = (JAWBONE_URL + 
       '?response_type=code&client_id='+ CLIENT_ID + 
       '&scope=' + PERMISSIONS +
       '&redirect_uri=' + REDIRECT_URI )

# The code below is necessary to pop up the login window from Jawbone
# The login still start in a separate thread, in order to allow the execution
# to continue, and run the "start_server()" call below.
threading.Timer(1, lambda: display(JS("window.open('%s')" % url))).start()

# And now start the webserver so that we can receive the answer from Jawbone API
start_server()

<IPython.core.display.Javascript object>

In [3]:
# Read the access token from the file
f = open(OAUTH_FILE, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)

auth_info

{u'access_token': u'aV1SI82xvTr8kBFGA3_2VAo-F5IbiT1zPxpUR_-PPzNC7LrLTyKlES6pRlVsJUdJ8c9QkoSzRYiFw6yPEhcRiFECdgRlo_GULMgGZS0EumxrKbZFiOmnmAPChBPDZ5JP',
 u'expires_in': 31536000,
 u'refresh_token': u'gkGL9TjxP67eAnSXM0K37py9XOpU5P3eut8Xi9a70xzdL2WxNgQvKekaCy5aBtavNNWfJhnfRQwlAN2iCODyqw',
 u'token_type': u'Bearer'}

At this point we are pretty much done. We only need to send standard API calls, as usual, with the only addition being that we include an extra _header_ in the request, with the access_token. The code below is just showing various use cases of the API.

In [4]:
# Testing one of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/bandevents"

# See https://jawbone.com/up/developer/endpoints/bandevents for details
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
bandevents_data = json.loads(resp.text)
bandevents_data

{u'data': {u'items': [{u'action': u'exit_sleep_mode',
    u'date': 20150811,
    u'time_created': 1439275648,
    u'tz': u'Europe/Athens'},
   {u'action': u'enter_sleep_mode',
    u'date': 20150811,
    u'time_created': 1439241728,
    u'tz': u'Europe/Athens'},
   {u'action': u'exit_sleep_mode',
    u'date': 20150810,
    u'time_created': 1439181184,
    u'tz': u'Europe/Athens'},
   {u'action': u'enter_sleep_mode',
    u'date': 20150810,
    u'time_created': 1439180288,
    u'tz': u'Europe/Athens'},
   {u'action': u'exit_sleep_mode',
    u'date': 20150810,
    u'time_created': 1439180288,
    u'tz': u'Europe/Athens'},
   {u'action': u'enter_sleep_mode',
    u'date': 20150810,
    u'time_created': 1439175936,
    u'tz': u'Europe/Athens'},
   {u'action': u'exit_sleep_mode',
    u'date': 20150810,
    u'time_created': 1439175680,
    u'tz': u'Europe/Athens'},
   {u'action': u'exit_sleep_mode',
    u'date': 20150727,
    u'time_created': 1438020608,
    u'tz': u'America/New_York'},
   {u'a

In [5]:
print bandevents_data[u'meta'][u'user_xid']

6CEDmZ-RtVRKxBB-NA5PoQ


In [6]:
import pandas as pd
df = pd.DataFrame(bandevents_data["data"]["items"])
df

,action,date,time_created,tz
0,exit_sleep_mode,20150811,1439275648,Europe/Athens
1,enter_sleep_mode,20150811,1439241728,Europe/Athens
2,exit_sleep_mode,20150810,1439181184,Europe/Athens
3,enter_sleep_mode,20150810,1439180288,Europe/Athens
4,exit_sleep_mode,20150810,1439180288,Europe/Athens
5,enter_sleep_mode,20150810,1439175936,Europe/Athens
6,exit_sleep_mode,20150810,1439175680,Europe/Athens
7,exit_sleep_mode,20150727,1438020608,America/New_York
8,enter_sleep_mode,20150727,1437974528,America/New_York
9,exit_sleep_mode,20150722,1437571840,America/New_York


In [7]:
df["time_created"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df["time_created"])

NameError: global name 'datetime' is not defined

In [ ]:
df

In [8]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/moves"

# See https://jawbone.com/up/developer/endpoints/moves for details
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
moves_data = json.loads(resp.text)
moves_data

{u'data': {u'items': [{u'date': 20150831,
    u'details': {u'active_time': 0,
     u'bg_calories': 4.64046832919,
     u'bmr': 754.959650969,
     u'bmr_day': 1754.84089644,
     u'calories': 4.64046832919,
     u'distance': 47.4755296707,
     u'hourly_totals': {u'2015083107': {u'active_time': 0,
       u'calories': 4.64046832919,
       u'distance': 47.4755296707,
       u'inactive_time': 0,
       u'longest_active_time': 0,
       u'longest_idle_time': 0,
       u'steps': 59}},
     u'inactive_time': 0,
     u'km': 0.0474755296707,
     u'longest_active': 0,
     u'longest_idle': 0,
     u'steps': 59,
     u'sunrise': 0,
     u'sunset': 0,
     u'tz': u'America/New_York',
     u'tzs': [[1441021397, u'America/New_York']],
     u'wo_active_time': 0,
     u'wo_calories': 0,
     u'wo_count': 0,
     u'wo_longest': 0,
     u'wo_time': 0},
    u'snapshot_image': u'/nudge/image/e/1441021906/trVF_JXR-jtOB1ClNquC7kuWnWM8m_Ko/F3rPnz_fTUY.png',
    u'time_completed': 1441021603,
    u'time_cr

In [ ]:
import pandas as pd

df = pd.DataFrame(moves_data["data"]["items"])
df

In [ ]:
df_details = pd.DataFrame(list(df["details"]))
df_details

In [ ]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/sleeps"

# See https://jawbone.com/up/developer/endpoints/sleeps
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
sleep_data = json.loads(resp.text)
sleep_data

In [ ]:
df = pd.DataFrame(sleep_data["data"]["items"])
df

In [ ]:
df_details = pd.DataFrame(list(df["details"]))
df_details

In [ ]:
df_details["asleep_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["asleep_time"])
df_details["awake_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["awake_time"])
df_details